# Example useage of ccs fp
This notebook functions as an example and documentation

Copyright IBM Corporation 2022.

SPDX-License-Identifier: MIT

In [ ]:
# Copyright IBM Corporation 2022.
# SPDX-License-Identifier: EPL-2.0

import ccsfp as cfp
import ccsfp.informatics as inf
import ccsfp.informatics.molecules_and_images as mai
import ccsfp.informatics.chemical_space_map as csm
import ccsfp.informatics.generation as gen
import ccsfp.informatics.finger_prints as fp

import rdkit
from rdkit import Chem
import numpy as np
import pandas as pd
import logging
logging.basicConfig(format='%(levelname)-9s : %(message)s')
log=logging.getLogger()
log.setLevel(logging.INFO)

To being we will create some random seed molecules. These molecules are given in smiles form below. We add to this other string identifiers InChI and InChIKeys. 

In [ ]:
smiles = ["NCCO", "N1CCNCC1", "CCCN", "CC(O)CN", "c1ccncc1", "CC(C=O)CC(O)CC1CCCCC1", "CC(C=O)CC(CCO)CC(C)CN(CCN)CNC"]
inchi = [inf.smiles.smiles_to_inchi(s) for s in smiles]
inchikey= [Chem.InchiToInchiKey(inch) for inch in inchi]

Next we create RDKit molecule objects from the smiles and give examples of how we can vary the output writh a few commonly used flags like adding hydrogen and generating 3D structures.

In [ ]:
mols = [mai.smiles_to_molecule(s, addH=True, threed=False) for s in smiles]
mols_no_h = [mai.smiles_to_molecule(s, addH=False, threed=False) for s in smiles]
mols3d = [mai.smiles_to_molecule(s, addH=True, threed=True) for s in smiles]

As an example we also have a property number of heavy atoms and label the molecules by numbers.

In [ ]:
num_heavy_atoms = [Chem.rdMolDescriptors.CalcNumHeavyAtoms(mol) for mol in mols]
labels = [Chem.rdMolDescriptors.CalcMolFormula(mol) for mol in mols]

To gather all of this together we build a pandas dataframe

In [ ]:
df = pd.DataFrame(data=np.array([smiles, inchi, inchikey, num_heavy_atoms, labels]).T, 
                  columns=["smiles", "inchi", "inchikey", "n_heavy_atoms", "names"])

We can now visualize the molecules to see what we have to start from. We do this with the 3D structure s and the 2D so we can visually see the difference.

In [ ]:
for m3d, m in zip(mols3d, mols):
    log.info("{} {}".format(display(m3d), display(m)))

## Generating new molecules
In this section we use of seed moelcules to generate new molecules using a non-exhaustive core and side chain addition. This process is contain in one function call. The process is deterministic but can different in the order in which structures are generated. The process can be altered in terms of number of structures generated if overwriting is False extra random is false and generation number is changed.

The method breaks down a list of smiles strings into cores (using Murko hashes) and stores them in a csv file. The method proceeds to break down another list of smiles into side chains (using regioisomer hashes) and stores them in a csv file. In each generation stage each core is chosen in order and a pseudo-random side chaiin is chosen. The two are connected either by the first attachement point on the core (noted by a `*`) or the first atom in the scaffold. If new attachment points are added to teh structure by a side chain a recursive call is made until the attachment points are filled. The algorithim is not fool proof has been shown to be effective.

In the example we give the same set of smiles for core and sidechain setting, these can be different.

In [ ]:
gensmiles = gen.limited_enumeration(smiles, 
                                    smiles, 
                                    generations=20, 
                                    extra_random=True, 
                                    overwrite=True, 
                                    return_all=False
                                   )

The smiles that have been generated are

In [ ]:
gensmiles

Now we can visualize these as well

In [ ]:
log.info("\nMade {} smiles\n".format(len(gensmiles)))

for gs in gensmiles:
    log.info(gs)
    log.info(display(mai.smiles_to_molecule(gs, threed=False)));

We can combine these with our input smiles and save them for later

In [ ]:
geninchi = [inf.smiles.smiles_to_inchi(s) for s in gensmiles]
geninchikey = [Chem.InchiToInchiKey(inch) for inch in geninchi]
gennum_heavy_atoms = [Chem.rdMolDescriptors.CalcNumHeavyAtoms(mai.smiles_to_molecule(s)) for s in gensmiles]
genlabels = [Chem.rdMolDescriptors.CalcMolFormula(mai.smiles_to_molecule(s)) for s in gensmiles]

gen_df = pd.DataFrame(np.array([gensmiles, geninchi, geninchikey, gennum_heavy_atoms, genlabels]).T,
                      columns=["smiles", "inchi", "inchikey", "n_heavy_atoms", "names"])
df = pd.concat([df, gen_df])
df.reset_index(inplace=True, drop=True)

In [ ]:
df

In [ ]:
df.to_csv("molecules.csv", index=False)

## CCS fingerprint generation
In this section we show to generate the CCS fingerprints from molecule string representations.

First lets load our data cleanly

In [ ]:
df = pd.read_csv("molecules.csv")

The ccs fingerprint is generated using the class `ccus_fps`. This class can provide an explanation of the fingerprint to help with interpretation.

In [ ]:
ccus_fp = fp.ccus_fps()
ccs_df = ccus_fp.get_fp_information()

Now we can generate our fingerprints from smiles or inchi in one command. The thresh is the threshold needed before the fingerprint runs in parallel.

In [ ]:
fps_smi, df_fps_smi, smarts_smi = fp.ccs_fp(df["smiles"].to_list(), 
                                            thresh=1000)
df_fps_smi

In [ ]:
fps_inchi, df_fps_inchi, smarts_inchi = fp.ccs_fp(df["inchi"].to_list(), 
                                                  thresh=1000)
df_fps_inchi

In [ ]:
try:
    pd.testing.assert_frame_equal(df_fps_smi, df_fps_inchi)
    log.info("Equal")
except AssertionError as aerr:
    log.info("Not Equal!")


You can also define your own smarts substructure patterns to search for and hence your own fingerprint.

In [ ]:
fps, df_fps, smarts = fp.ccs_fp(df["smiles"].to_list(),
                                            version=1, 
                                            substructures=["[CX3][C]", "[c,n]1[c,n][c,n][c,n][c,n][c,n]1"],
                                            substructure_names=["a", "b"])
df_fps

## Chemical space plotting
In this section we show how to produce a chemical space plot

First we will load the data clean

In [ ]:
df = pd.read_csv("molecules.csv")

One function call will plot the chemical space diagram and annotate it with the molecules pictures closest to the centroid specified. The diagram is produced using the Fruchterman-Reingold force-directed algorithm. Each node represents a molecule and nodes are connected if their tanimoto similarity is >= the connection threshold. Those with more connections will tend to stay in the centre of the diagram whilst those less connected move to the edges. The graph can be used to calculate average numbers of connections and other metrics.

In [ ]:
csm.plot_annotated_chemical_space(df, 
                                  prop_key="n_heavy_atoms", 
                                  smiles_key="smiles",
                                  label_key="names",
                                  centroid=(0.7, 0.4), 
                                  closest_n=4,
                                  connection_threshold=0.7
                                 )

## Removing duplicates
In this section we give an example of removing duplicate entries from our data. We compare all string identifiers and can chose which to use to remove ducplicates. 

First lets start with a clean load our data

In [ ]:
df = pd.read_csv("molecules.csv")

In [ ]:
dup_indexes = cfp.utilities.utility_functions.check_for_duplicates(data=df,  
                                                              compare_which="inchikey")
log.info("Duplicate index list: {}".format(dup_indexes))

In [ ]:
df.drop(index=dup_indexes, inplace=True)

In [ ]:
df

In [ ]:
df.to_csv("unique_molecules.csv", index=False)